In [11]:
from keras.datasets import mnist 
import matplotlib.pyplot as plt 
from keras.optimizers import Adam 
from keras import utils 
import numpy as np
import matplotlib.pyplot as plt 
from tensorflow.keras.models import load_model 

dataset = mnist
(x_train, y_train), (x_test, y_test) = dataset.load_data() #translate the data to image 

x_test = x_test.reshape(x_test.shape[0], -1).astype("float64")

#normalization 
x_test /= 255

x_test = x_test.reshape(-1, 28, 28, 1)

lenet5 = load_model('Model/lenet5_mnist.h5')#load lenet5 model trained with mnist dataset 
lenet5.compile(optimizer=Adam(),loss='sparse_categorical_crossentropy',metrics=['accuracy']) 

def sample(x_test,y_test,N,beta):
    
    max_distance = 0  #max safe distance
    count = 0
    count2 = 0
    neighbour_list=[]
    mean = np.zeros(2)  
    cov = np.eye(2)
    label = y_test
    
    while True:
        count2+=1
        p = np.random.multivariate_normal(mean, cov,(28,28)) #sample formula
        p = p.reshape(-1,28,28,1) 
        neighbour = p[0] + x_test*255
        neighbour /= 255
        distance=np.linalg.norm(p[0]) 
        distance /= 255
        #print(distance)
        if (distance < beta or distance == beta):
            #print(distance)
            output = lenet5.predict(neighbour.reshape(-1, 28, 28, 1),batch_size=128)[0]
            proba = softmax(output)
            pred = np.argmax(proba)
            neighbour = (neighbour*255).astype(np.uint8).reshape(28,28)
            if(pred == label): 
                neighbour_list.append((proba[label],neighbour))
                count+=1  #correct prediction
            else:  #if there are some iamges inside beta cannot have the same label, beta will be decreased
                beta = beta-0.0001
                neighbour_list.append((proba[label],neighbour))
                print("Beta is reduced by 0.0001, the new beta is",beta)   
            
        if(count==N):  
            print("Sampled %s images inside beta" % count)
            print("%s images inside beta have the same label, local robustness formula satisfied" %N)
            break

    return beta

def softmax(x):
    e_x = np.exp(x-np.max(x))
    return e_x / e_x.sum()

In [14]:
beta = 0.0987
N = 783
sample(x_test[1],y_test[1],N, beta)

Sampled 783 images inside beta
783 images inside beta have the same label, local robustness formula satisfied


0.0987

In [13]:
beta = 0.1101
N = 671
sample(x_test[1],y_test[1],N, beta)

Sampled 671 images inside beta
671 images inside beta have the same label, local robustness formula satisfied


0.1101